# Gerador de Documentação DBT - Conexa

 - Time: Engenheira de Dados
 - Autor: Anselmo e David
 - Esse script cria o arquivo model do DBT a partir dos prompts abaixo
 - 1° ele cria um arquivo de descrição com as amostras dos dados e descrição das colunas
 - 2° utiliza o arquivo do step anterior para incluir no contexto do modelo LLM e assim implementar a RAG
 - 3° Executa a prompt para criar um arquivo DBT válido, utiliza o contexo de todos os dados e gerar uma documentação mais completa.


In [ ]:
# Bibliotecas
from dotenv import load_dotenv
from sqlalchemy import create_engine, MetaData
from sqlalchemy import inspect
from trino.auth import BasicAuthentication
import pandas as pd
import os
load_dotenv(dotenv_path='./../.env')

In [ ]:
#key = os.getenv('GROQ_API3')
key_trino = os.getenv('KEY_TRINO')
key_gpt= os.getenv('OPENAI_API_KEY')
url_engine_postgres = os.getenv('ENGINE_POSTGRES')

In [ ]:
schemas =  ['gold', 'silver', 'bronze']

In [ ]:
schema = 'gold'
modelo = "llama3-70b-8192"
modelo_hf_emb="BAAI/bge-m3"

In [ ]:
engine_postgres = create_engine(url_engine_postgres)

In [ ]:
from llama_index.core import Settings
from llama_index.llms.groq import Groq
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core import SQLDatabase
from llama_index.core.objects import SQLTableNodeMapping
from llama_index.core.objects import SQLTableSchema
from llama_index.core.objects import ObjectIndex
from llama_index.core import VectorStoreIndex
from llama_index.core.indices.struct_store.sql_query import SQLTableRetrieverQueryEngine

In [ ]:
Settings.llm = OpenAI(
    model="gpt-4o-mini",
    api_key=key_gpt,
)
Settings.embed_model = HuggingFaceEmbedding(model_name = modelo_hf_emb)

In [ ]:
def gerar_descricao_tabela(nome_tabela, df_amostra, schema):
    prompt = f"""
    Analise a amostra da tabela: '{nome_tabela}' forneça uma curta e breve descrição do conteúdo dessa tabela.
    Em seguida faça a descrição de todas as colunas baseado nas amostras abaixo.
    Informe até o máximo de 5 valores únicos de cada coluna e o tipo.

    Amostra da Tabela:
    {df_amostra}

    Descrição:
    """

    resposta = Settings.llm.complete(prompt = prompt)
    conteudo_formatado = resposta.text

    output_dir = f"../descricao_{schema}/"

    os.makedirs(output_dir, exist_ok=True)
    with open(output_dir + f'{nome_tabela}_descrição.yaml', 'w', encoding='utf-8') as arquivo:
        arquivo.write(conteudo_formatado)
    return resposta.text

In [ ]:
def cria_dicionario_tabela(nome_tabela, engine, schema):
  output_dir = f"../descricao_{schema}/"
  dicionario_tabelas = {}
  df = pd.read_sql_query(f'select * from {schema}.{nome_tabela} limit 10', engine)
  df_amostra = df.head(10).to_dict('records')
  if os.path.exists(output_dir + f'{schema}.{nome_tabela}_descrição.txt'):
    with open(output_dir + f'{schema}.{nome_tabela}_descrição.txt', 'r', encoding='utf-8') as arquivo:
      descricao = arquivo.read()
  else:
    descricao = gerar_descricao_tabela(nome_tabela, df_amostra=df_amostra, schema=schema)
  dicionario_tabelas[nome_tabela] = descricao
  return dicionario_tabelas

In [ ]:
def resposta_prompt_2(tabela, query_engine_2: SQLTableRetrieverQueryEngine):
    prompt_2 =  f"""
Criei um arquivo DBT do tipo model (não source) e quero incluir as descrições (description) no bloco YAML dentro do arquivo schema.yml, referenciando a tabela: "{tabela}".

Gere uma descrição técnica clara e objetiva com base no nome e na amostra de dados de cada coluna.

A saída deve seguir o padrão abaixo:

"version: 2

models:
  - name: <nome_da_tabela>
    description: <descrição geral da tabela, baseada no nome>
    columns:
      - name: <nome_coluna>
        description: <descrição inferida da coluna com base no nome e nos exemplos>
"

Adote uma linguagem apropriada para analistas de dados e engenheiros de dados.
Gere as descrições para todas as colunas que existem na tabela: {tabela}.
Todas as respostas devem ser em português PT-BR.
Os campos que não forem possivel determinar preencha como: Sem Descrição
Não incluir caracteres: ":" nas descrições
Apresente apenas o código DBT, Não inclua textos de decrição, apresentação, conclusão e markdown.
"""
    resposta = query_engine_2.query(prompt_2)
    
    return resposta

In [ ]:
def cria_arquivo_na_pasta(tabela, resposta, output_dir):
    conteudo_formatado = resposta.response.replace('\\n', '\n')
    os.makedirs(output_dir, exist_ok=True)
    with open(output_dir + f'{tabela}.yaml', 'w', encoding='utf-8') as arquivo:
        arquivo.write(conteudo_formatado)

In [ ]:
for schema in schemas:

    inspector = inspect(engine_postgres)
    tabelas_trino = inspector.get_table_names(schema=schema)

    output_dir = f"../{schema}/"

    for tabela in tabelas_trino:
        if os.path.exists(output_dir + f'{tabela}.yaml'):
            print(f"🔄 O arquivo yaml {schema}.{tabela} já existe, avançando...")
        else:
            sql_database = SQLDatabase(engine_postgres, schema=schema)
            table_node_map = SQLTableNodeMapping(sql_database)

            dicionario_tabelas = cria_dicionario_tabela(tabela, engine_postgres, schema=schema)
            #Atualiza o schema da tabela
            table_schema_objs = [
                SQLTableSchema(table_name= tabela, context_str= dicionario_tabelas[tabela])
            ]

            obj_index = ObjectIndex.from_objects(table_schema_objs, table_node_map, VectorStoreIndex)
            obj_retriever = obj_index.as_retriever(similarity_top_k=1)

            query_engine_2 = SQLTableRetrieverQueryEngine(sql_database, obj_retriever)
            resposta = resposta_prompt_2(tabela, query_engine_2)
            cria_arquivo_na_pasta(tabela, resposta, output_dir)
            print(f"🔄 O arquivo yaml {schema}.{tabela} foi criado, avançando...")